<h1>Anomaly detection in time series using logistic regression</h1>

In [1]:
import numpy as np
import pandas as pd

In [7]:
df = pd.read_csv('../data/resurfacing_marking_data.csv')

In [16]:
# sine-cosine transformation of time features 
df['time'] = pd.to_datetime(df['time'])

# component minute
df['minite_sin'] = np.sin(2 * np.pi * df['time'].dt.minute / 60)
df['minite_cos'] = np.cos(2 * np.pi * df['time'].dt.minute / 60)

# component hour
df['hour_sin'] = np.sin(2 * np.pi * df['time'].dt.hour / 24)
df['hour_cos'] = np.cos(2 * np.pi * df['time'].dt.hour / 24)

# component day_of_week
df['dow_sin'] = np.sin(2 * np.pi * df['time'].dt.day_of_week / 7)
df['dow_cos'] = np.cos(2 * np.pi * df['time'].dt.day_of_week / 7)

In [47]:
seq_length = 60
step_size = 30
X = []
y = []

for i in range(len(df) - seq_length - step_size + 1):
    X.append(df.iloc[i: i + seq_length][[col for col in df.columns if col != 'time']].values)
    y.append(df.iloc[i + seq_length + step_size - 1]['flood'])

X = np.array(X).reshape(len(X), -1)
y = np.array(y).reshape(-1, 1)

In [50]:
from sklearn.linear_model import LogisticRegression

(39911, 1)